In [3]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
# 이미지 데이터셋, 전처리 모듈
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
# 데이터 전처리
transform = transforms.Compose([
    transforms.Grayscale(),          # 이미지를 흑백으로 변환
    transforms.ToTensor(),
    transforms.Resize((50, 50))      # 사이즈 50, 50으로 고정
])
# 데이터셋 로드
# train 데이터셋
train_img_root = '../seungmin/DATA/Dataset/train/'
train_dataset = ImageFolder(root=train_img_root, transform=transform)

# test 데이터셋
test_img_root = '../seungmin/DATA/Dataset/test/'
test_dataset = ImageFolder(root=test_img_root, transform=transform)
train_dataset
# class 즉 타겟 확인
train_dataset.classes
# 한 번만 돌려서 형태 확인
for feature, target in train_dataset:
    print(feature.shape, target)
    break
# CNN 모델 만들기
class VehicleCNN(nn.Module):
      
    def __init__(self, num_classes=3):  # num_classes는 분류하고자 하는 클래스가 육, 해, 공 3종류이므로 3으로 설정
        super(VehicleCNN, self).__init__()
        # 첫 번째 합성곱 레이어: 입력 채널 1, 출력 채널 64, 커널 크기 3, 스트라이드 1, 패딩 'same'
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding='same')   # 입력 채널 1 (grayscale 이미지) -> 최후수단 padding='same'
        # 최대 풀링 레이어: 커널 크기 2, 스트라이드 2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # 첫 번째 드롭아웃 레이어: 드롭아웃 비율 0.25
        self.dropout1 = nn.Dropout(0.25)
        
        # 두 번째 합성곱 레이어: 입력 채널 64, 출력 채널 128, 커널 크기 3, 스트라이드 1, 패딩 'same'
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding='same') # 위에서 out_channels 64-> 32-> 16 -> 8 ->10 -> 64 ->128로 바꿈 
        # 두 번째 드롭아웃 레이어: 드롭아웃 비율 0.25
        self.dropout2 = nn.Dropout(0.25)

        # 세 번째 합성곱 레이어: 입력 채널 64, 출력 채널 128, 커널 크기 3, 스트라이드 1, 패딩 'same'
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding='same') # 위에서 out_channels 64-> 32-> 16 -> 8 ->10 -> 64 ->128로 바꿈 
        # 세 번째 드롭아웃 레이어: 드롭아웃 비율 0.25
        self.dropout3 = nn.Dropout(0.25)
        
        # 첫 번째 완전 연결 레이어: 입력 특성 128*12*12, 출력 특성 50
        self.fc1 = nn.Linear(128 * 6 * 6, 50)                       # 120 -> 60 -> 30 -> 50
        # 세 번째 드롭아웃 레이어: 드롭아웃 비율 0.5
        self.dropout4 = nn.Dropout(0.5)
        
        # 두 번째 완전 연결 레이어: 입력 특성 50, 출력 특성 num_classes
        self.fc2 = nn.Linear(50, num_classes)
        
    def forward(self, x):
        # 첫 번째 합성곱 적용
        x = self.conv1(x)
        # 활성화 함수 ReLU 적용
        x = F.relu(x)
        # 최대 풀링 적용
        x = self.pool(x)
        # 첫 번째 드롭아웃 적용
        x = self.dropout1(x)

        # 두 번째 합성곱 적용
        x = self.conv2(x)
        # 활성화 함수 ReLU 적용
        x = F.relu(x)
        # 최대 풀링 적용
        x = self.pool(x)
        # 두 번째 드롭아웃 적용
        x = self.dropout2(x)

        # 세 번째 합성곱 적용
        x = self.conv3(x)
        # 활성화 함수 ReLU 적용
        x = F.relu(x)
        # 최대 풀링 적용
        x = self.pool(x)
        # 세 번째 드롭아웃 적용
        x = self.dropout3(x)

        # 특성 벡터로 변환 (평탄화)
        x = x.view(x.size(0), -1)

        # 첫 번째 완전 연결 레이어 적용
        x = self.fc1(x)
        # 활성화 함수 ReLU 적용
        x = F.relu(x)
        # 세 번째 드롭아웃 적용
        x = self.dropout4(x)

        # 두 번째 완전 연결 레이어 적용
        x = self.fc2(x)

        return x
# 학습 준비
import torch.nn.functional as F

# 모델 인스턴스 생성
model = VehicleCNN(num_classes=len(train_dataset.classes))

optimizer = optim.Adam(model.parameters(), lr=0.001) # --> vald 값이 조절이 일정수치 이하로 안 떨어지면 lr값 변화 시도 (0.001 -> 0.1)
cost = nn.CrossEntropyLoss()

EPOCHS = 100           # 50 -> 30-> 100 (다시 늘린 이유 : 30에서 에포크 횟수가 늘어날수록 loss값이 아름답게 줄어들어서)
stopCall = 10          # 5 -> 10으로 변경
# 스케쥴러 설정
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)  # 30 EPOCHS마다 학습률을 0.1배 감소
torch.random.manual_seed(12)
train_dataset, val_dataset = random_split(train_dataset, [0.7,0.3])
# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=30, shuffle=True)   # shuffle=True 는 무작위 추출
val_loader = DataLoader(val_dataset, batch_size=30, shuffle=True)      # 마찬가지(시계열 데이터가 아니므로 무작위 설정) / 과대적합 방지로 val_loader
# 모델 학습
best_val_loss = float('inf')  # 최적의 검증 손실값을 저장할 변수 초기화
earlystop = 0                 # 조기 종료 카운터 초기화

for epoch in range(EPOCHS):
    model.train()             # 모델을 학습 모드로 설정
    loss_sum = 0.0
    
    for images, labels in train_loader:
        optimizer.zero_grad()           # 옵티마이저의 그라디언트 초기화
        outputs = model(images)         # 모델에 입력을 주고 출력을 얻음
        loss = cost(outputs, labels)    # 출력과 타겟 라벨 사이의 손실 계산
        loss.backward()                 # 손실에 대한 모델의 그라디언트 계산
        optimizer.step()                # 옵티마이저를 사용하여 파라미터 업데이트
        
        loss_sum += loss.item() * images.size(0)
    
    loss_avg = loss_sum / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{EPOCHS}], Loss: {loss_avg:.4f}')
    
    # 검증 단계
    model.eval()                        # 모델을 평가 모드로 설정
    val_loss_sum = 0.0
    
    with torch.no_grad():               # 그라디언트 계산을 하지 않음
        for images, labels in val_loader:
            outputs = model(images)
            loss = cost(outputs, labels)
            val_loss_sum += loss.item() * images.size(0)
    
    val_loss_avg = val_loss_sum / len(val_loader.dataset)
    print(f'Validation Loss: {val_loss_avg:.4f}')
    
    # 스케줄러 단계 진행
    scheduler.step(val_loss_avg)
    
    # 모델 체크포인트와 조기 종료 검사, 제일 좋은 모델 저장
    if val_loss_avg < best_val_loss:
        best_val_loss = val_loss_avg
        torch.save(model.state_dict(), 'best_model.pth')
        earlystop = 0
    else:
        earlystop += 1

    if earlystop > stopCall:
        print("조기종료")
        break

model.eval()      # 모델을 평가 모드로 설정
final_loss = 0    # 예측값과 라벨 사이의 손실 계산 
val_correct = 0       # 정답

with torch.no_grad():  # 평가시 모델 업데이트 불필요 / 그라디언트 계산을 하지 않음
    for images, labels in val_loader:
        #print(images.shape)
        outputs = model(images)
        final_loss += cost(outputs, labels).item()  # 손실을 누적
        pred = outputs.argmax(dim=1, keepdim=True)  # 가장 높은 값을 가진 인덱스를 예측값으로 선택
        val_correct += pred.eq(labels.view_as(pred)).sum().item()

final_loss /= len(val_loader.dataset)
print(f'평균 손실: {final_loss:.4f}, Accuracy: {val_correct}/{len(val_loader.dataset)} ({100. * val_correct / len(val_loader.dataset):.0f}%)')
# DataLoader 설정
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=True)   # shuffle=True 는 무작위 추출
model.eval()      # 모델을 평가 모드로 설정
test_loss = 0    # 예측값과 라벨 사이의 손실 계산 
test_correct = 0       # 정답

# with torch.no_grad():  # 평가시 모델 업데이트 불필요 / 그라디언트 계산을 하지 않음
#     for images, labels in test_loader:
#         #print(images.shape)
#         outputs = model(images)
#         test_loss += cost(outputs, labels).item()  # 손실을 누적
#         pred = outputs.argmax(dim=1, keepdim=True)  # 가장 높은 값을 가진 인덱스를 예측값으로 선택
#         test_correct += pred.eq(labels.view_as(pred)).sum().item()

# test_loss /= len(test_loader.dataset)
# print(f'평균 손실: {test_loss:.4f}, Accuracy: {test_correct}/{len(test_loader.dataset)} ({100. * test_correct / len(test_loader.dataset):.0f}%)')
# 실제 예측
from torchvision import transforms
from PIL import Image

# 이미지 전처리를 위한 변환 정의
transform = transforms.Compose([
    transforms.Grayscale(),          # 이미지를 흑백으로 변환
    transforms.Resize((50, 50)),     # 이미지 크기 조정
    transforms.ToTensor()            # 이미지를 PyTorch 텐서로 변환
])

model_pth = '../seungmin/best_model.pth'  # best model의 주소
# 학습된 모델을 로드하는 함수
def load_trained_model(model_pth):
    model = VehicleCNN(num_classes=3)  
    model.load_state_dict(torch.load(model_pth))
    model.eval()  # 모델을 평가 모드로 설정
    return model
img_pth = '../seungmin/DATA/pre/tank.jpg'
# 이미지를 입력으로 받아 예측을 수행하는 함수
def predict_image(img_pth, model, transform):
    image = Image.open(img_pth)  # 이미지 파일을 열기
    image = transform(image)        # 이미지 전처리
    image = image.unsqueeze(0)      # 배치 차원 추가 ([1, C, H, W] 형태로 변환)
    with torch.no_grad():  # 그라디언트 계산을 하지 않음
        output = model(image)  # 모델을 통해 예측 수행
        prediction = output.argmax(dim=1)  # 가장 높은 점수를 가진 클래스 선택
    return prediction.item()


model = load_trained_model(model_pth)
predicted_class = predict_image(img_pth, model, transform)

what = ''
if predicted_class == 0:
    what = 'airplanes'
elif predicted_class == 1:
    what = 'cars'
elif predicted_class == 2:
    what = 'ship'
print(f'예측 결과 : {what}')


FileNotFoundError: [Errno 2] No such file or directory: '../seungmin/DATA/Dataset/train/'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import torchvision.models as models
# 데이터 전처리
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomRotation(degrees=15),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Resize((300, 300))
])
# 데이터셋 로드
img_root = 'data'
dataset = ImageFolder(root=img_root, transform=transform)

# 데이터셋 분할 (학습/검증/테스트)
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# 하이퍼파라미터 설정
batch_size = 32
num_epochs = 300
learning_rate = 0.001
# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# VGG 모델 로드 및 전이학습 설정
vgg = models.vgg16(pretrained=True)

# VGG 첫 번째 컨볼루션 레이어의 입력 채널 수 수정
vgg.features[0] = nn.Conv2d(1, 64, kernel_size=5, padding=1)

for param in vgg.features.parameters():
    param.requires_grad = False


num_features = vgg.classifier[0].in_features
vgg.classifier = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, len(dataset.classes))
)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
# 모델 설정
model = vgg.to(DEVICE)

# 손실 함수 정의
criterion = nn.CrossEntropyLoss()

# 옵티마이저 정의
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 리스트 정의
trainList = []
valList = []
testList = []
# 학습 함수
def train(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(loader, 0):
        inputs, labels = data[0].to(DEVICE), data[1].to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        trainList.append(loss)
    return running_loss / len(loader)
# 검증 함수
def validate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(loader, 0):
            inputs, labels = data[0].to(DEVICE), data[1].to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            valList.append(loss)
    return running_loss / len(loader)
from torch.optim.lr_scheduler import ReduceLROnPlateau
SCHEDULERS = ReduceLROnPlateau(optimizer, mode = 'min', patience = 3, verbose = True)
cnt = 0

# # 학습 루프
# best_val_loss = 10000
# for epoch in range(num_epochs):
#     train_loss = train(model, train_loader, optimizer, criterion)
#     val_loss = validate(model, val_loader, criterion)
#     print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
#     if val_loss < best_val_loss:
#         best_val_loss = val_loss
#         torch.save(model.state_dict(), 'best_model.pth')
#     if SCHEDULERS.num_bad_epochs >= SCHEDULERS.patience:
#         print(f"Early stopping at epoch {epoch}")
#         break
#     # print(epoch)
# import matplotlib.pyplot as plt

# 에포크 loss 시각화
plt.figure(figsize=(8, 6))
x = range(1,len(trainList)+1)
plt.plot(x, torch.tensor(trainList).detach().numpy(), label='Train Loss')
x = range(1,len(valList)+1)
plt.plot(x, torch.tensor(valList).detach().numpy(), label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train and Validation Loss')
plt.legend()
plt.show()


# 테스트 함수
def test(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            inputs, labels = data[0].to(DEVICE), data[1].to(DEVICE)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    testList.append(accuracy)
    return accuracy


# 모델 로드 및 테스트
model.load_state_dict(torch.load('best_model.pth'))
test_accuracy = test(model, test_loader)
print(f'Test Accuracy: {test_accuracy:.2f}%')

from PIL import Image

# 테스트 결과 출력

img_path = 'test/image2.png'  # 테스트 이미지 경로

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # 그레이스케일 변환
    transforms.ToTensor(),
    transforms.Resize((300, 300))
])

img = Image.open(img_path).convert('L')
img_tensor = transform(img).unsqueeze(0).to(DEVICE)

model.eval()

with torch.no_grad():
    output = model(img_tensor)
    _, predicted = torch.max(output.data, 1)

plt.imshow(img, cmap='gray')  # 그레이스케일 이미지로 표시
plt.axis('off')
plt.show()

# 결과 출력
prob = torch.softmax(output, dim=1).detach().cpu().squeeze().numpy()
max_prob = max(prob)
rounded_prob = round(max_prob, 2)

class_names = ["ev6","ev9","qm3","qm6","sm3","sm6","test","xm3","그랜저","넥쏘","더 뉴 아반떼","렉스턴","스타리아","싼타페","쏘나타","아우디 a4","아우디 a5","아우디 a6","아우디 a7","아우디 e-tron","아우디 q3","아우디 q7","아우디 q8","아이오닉5","아이오닉6","제네시스 g70","제네시스 g90","제네시스 gv70","코나","코란도","토레스","투싼"]  # 클래스 이름 리스트 (실제 클래스 이름으로 변경해야 함)
predicted_class = class_names[predicted.item()]

print(f"예측 클래스: {predicted_class}")
print(f"예측 확률: {rounded_prob*100:.2f}%")
class_names = ["test",]  # 클래스 이름 리스트 (실제 클래스 이름으로 변경해야 함)
brandList = {
    "현대" : ["그랜저","더 뉴 아반떼","스타리아","아이오닉5","아이오닉6","넥쏘","쏘나타","투싼", "코나","싼타페",],
    "기아" : ["ev6","ev9",],
    "르노" : ["qm3","qm6","sm3","sm6","xm3",],
    "제네시스" : ["제네시스 g70","제네시스 g90","제네시스 gv70",],
    "아우디"  : ["아우디 a4","아우디 a5","아우디 a6","아우디 a7","아우디 e-tron","아우디 q3","아우디 q7","아우디 q8",],
    "KG 모빌리티" : ["렉스턴","코란도","토레스",]
}

# 결과 출력
prob = torch.softmax(output, dim=1).detach().cpu().squeeze().numpy()
max_prob = max(prob)
rounded_prob = round(max_prob, 2)

class_names = ["ev6","ev9","qm3","qm6","sm3","sm6","test","xm3","그랜저","넥쏘","더 뉴 아반떼","렉스턴","스타리아","싼타페","쏘나타","아우디 a4","아우디 a5","아우디 a6","아우디 a7","아우디 e-tron","아우디 q3","아우디 q7","아우디 q8","아이오닉5","아이오닉6","제네시스 g70","제네시스 g90","제네시스 gv70","코나","코란도","토레스","투싼"]  # 클래스 이름 리스트 (실제 클래스 이름으로 변경해야 함)
predicted_class = class_names[predicted.item()] 


for brand, models in brandList.items():
    if predicted_class in models:
        print(f"예측 브랜드: {brand}")
        break

print(f"예측 클래스: {predicted_class}")
print(f"예측 확률: {rounded_prob*100:.2f}%")



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchmetrics.functional as metrics
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from bs4 import BeautifulSoup
import os
from torchinfo import summary
from torchvision.models import resnet18
# 군용 항공기 종류 식별자
AIRCRAFT_TYPE = {
    'A':'Attater',
    'B':'Bomber',
    'C':'Cargo',
    'F':'Fighter',
    'AH':'Attack Helicopter',
    'CH':'Cargo Helicopter',
    'UH':'Utility Helicopter'
}
# 군용 항공기 제조국 식별자
AIRCRAFT_NATION = [
    'USA',
    'EUROPE',
    'RUSSIA',
    'CHINA'
]
# 군용 항공기 명명 식별자
AIRCRAFT_NAME = [
    # USA
    'A-10',
    'B-1',
    'B-2',
    'B-52',
    'C-5',
    'C-17',
    'C-130',
    'CV-22',
    'F-15',
    'F-16',
    'FA-18',
    'F-22',
    'F-35',
    'AH-1',
    'AH-64',
    'CH-46',
    'CH-47',
    'CH-53',
    'UH-60',
    # Europe
    'EF2000',
    'Rafale',
    'JAS-39',
    'AV-8',
    # Russia
    'MIG-29',
    'MIG-31',
    'MIG-35',
    'Su-30',
    'Su-34',
    'Su-57',
    'Tu-160',
    'Mi-8',
    'Mi-24',
    'Mi-28',
    'Ka-52',
    # CHINA
    'J-15',
    'J-20',
    'H-6',
    'Y-20',
    'WZ-10',
    # Unidentified
    'UNIDENTIFIED'
]
len(AIRCRAFT_NAME)
torch.random.manual_seed(40)
data_path = './data/2/'

for name in AIRCRAFT_NAME:
    os.makedirs(os.path.join(data_path, name), exist_ok=True)
transformer = transforms.Compose(transforms=[
    transforms.ToTensor(),
    transforms.Resize(size=[96, 96])]
)
root_path = './data/'

ImgDS_1 = ImageFolder(root_path+'1/', transform=transformer)
ImgDS_2 = ImageFolder(root_path+'2/', transform=transformer)
for f, t in ImgDS_1:
    print(f.shape, t)
    break
set(ImgDS_1.targets)
set(ImgDS_2.targets)
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(3, 128, 3, padding='same'),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(128, 256, 3, padding='same'),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            
            nn.Conv2d(256, 256, 3, padding='same'),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(256, 256, 3, padding='same'),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            
            nn.Conv2d(256, 128, 3, padding='same'),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            nn.Conv2d(128, 128, 3, padding='same'),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            
            nn.Flatten(),
            nn.Linear(128*12*12, num_classes)
        )
        
    def forward(self, x):
        return self.layers(x)
def makeLoader(self, batch=1, sampler=None):
    train, valid, test = random_split(self, [0.7, 0.1, 0.2],
                                      torch.Generator().manual_seed(40))
    trainDL = DataLoader(train, batch_size=batch, sampler=sampler, shuffle=True)
    validDL = DataLoader(valid, batch_size=batch, sampler=sampler, shuffle=True)
    testDL = DataLoader(test, batch_size=batch, sampler=sampler, shuffle=True)
    
    return trainDL,validDL,testDL
train_cost_list = []
train_acc_list = []
valid_cost_list = []
valid_acc_list = []
def train_model(model, opt, epochs, trainDL, validDL, schd=None):
    for epoch in range(1, epochs+1):
        for f, t in trainDL:
            model.train()
            sum_acc = 0
            
            h = model(f)
            print(f.shape, t.shape, h.shape)
            
            cost = F.cross_entropy(h, t)
            
            opt.zero_grad()
            cost.backward()
            opt.step()
            
            sum_acc += metrics.accuracy(h, t, task='multiclass', num_classes=model.layers[-1].out_features)
            # print('.', end='')
            
        train_cost_list.append(cost)
        train_acc_list.append(sum_acc / len(trainDL))
        
        for f, t in validDL:
            model.eval()
            sum_acc = 0
            
            h = model(f)
            
            cost = F.cross_entropy(h, t)
            
            sum_acc += metrics.accuracy(h, t, task='multiclass', num_classes=model.layers[-1].out_features)
            
        valid_cost_list.append(cost)
        valid_acc_list.append(sum_acc / len(validDL))
        
        print(f'Epoch [{epoch:4}/{epochs:4}] ----------')
        print(f'Train cost : {train_cost_list[-1]}, Train acc : {train_acc_list[-1]}')
        print(f'Valid cost : {valid_cost_list[-1]}, Valid acc: {valid_acc_list[-1]}')
        
        if schd is not None:
            schd.step(valid_cost_list[-1])
        
def test_model(model, testDL):
    test_acc_list = []
    test_f1_list = []
    
    model.eval()
    for f, t in testDL:
        h = model(f)
        
        test_acc_list.append(metrics.accuracy(h, t, task='multiclass', num_classes=model.layers[-1].out_features))
        test_f1_list.append(metrics.f1_score(h, t, task='multiclass', num_classes=model.layers[-1].out_features))
        
    print(f'Average Test acc : {sum(test_acc_list) / len(testDL)}')
    print(f'Average Test f1  : {sum(test_f1_list) / len(testDL)}')
def exportModel(model, filename):
    pass
def model_predict(model, input):
    pass
trainDL1, validDL1, testDL1 = makeLoader(ImgDS_1, 64)
trainDL2, validDL2, testDL2 = makeLoader(ImgDS_2, 64)
model1 = CNN(len(os.listdir('./data/1/')))
model1.layers[-1].out_features
model2 = CNN(len(AIRCRAFT_NAME))
summary((model1))
opt1 = optim.Adam(model1.parameters())
opt2 = optim.Adam(model2.parameters())
schd1 = optim.lr_scheduler.ReduceLROnPlateau(opt1)
schd2 = optim.lr_scheduler.ReduceLROnPlateau(opt2)
# train_model(model2, opt2, 50, trainDL2, validDL2, schd2)
# torch.save(model2.state_dict(), 'H_best_model2.pth')

# summary(model1)
# resNet_1 = resnet18()
# resNet_1.lay
plt.plot([c.item() for c in train_cost_list])
plt.plot([c.item() for c in valid_cost_list])
plt.legend(['train', 'valid'])
plt.title('Model2 Cost')
plt.show()
